In [1]:
import os
import re
import boto3
import json

import pandas as pd
import numpy as np

from sagemaker.session import Session

# Extract Asset and Liability Line Items
**We divide asset and liability line items according to a regex parse**

In [2]:
def bsSplit(array: np.ndarray) -> tuple:
    """
    Function splits an array by bisection, into asset and liability & equity terms. Assumes that line items
    are recorded according to standard accounting practices in orientation. 
    ------------------------------------------------------------------------------------
    :param: (type numpy.ndarray)
        An array of balance sheet line items for a given broker-dealer
        
    :return: (type tuple)
        Return a tuple of arrays; index 1 is the asset items, index 2 is the liability & equity items and
        index 3 is the index where the split occured
        
    NOTE: We make the assumption that liability line items always fall below asset line items and each 
          balance sheet has both asset and liability line items. If either are missing we avoid balance sheet
    """
    stop_idx1 = 0
    stop_idx2 = array.size        # default to length of line items array
    
    asset_idx = 0        # asset and liability identifiers
    liable_idx = 0
    
    # iterate through the line items as provided by the array
    for i, item in enumerate(array):
        # search string for presence of word 'assets' and liabilites
        val1 = re.search('assets', item, flags=re.I)
        val2 = re.search('liability|liabilities', item, flags=re.I)
        
        # if we find the term "asset" we count this index
        if val1 is not None:
            asset_idx = i + 1    
        
        # if we find the term "liability" we count this index
        if val2 is not None:
            liable_idx = i + 1
        
        # in the event we find an asset and liability term, we check to see if the asset index appears before
        # the liability index, this is to prevent the JP Morgan 2012/2013 Textract Error
        if (asset_idx != 0) and (liable_idx != 0):
            
            if asset_idx < liable_idx:
                stop_idx1 = asset_idx
                stop_idx2 = liable_idx
                
    # we should always keep track of the asset term (this is our primary splitter)
    if (asset_idx != 0) and (liable_idx == 0):
        stop_idx1 = asset_idx

    # check the very last, in event our liability term created an early cut-off (e.g. 42352-2003-01-28)
    if (val1 is None) & (val2 is None):
        stop_idx2 = array.size
    
    # partition the array by the enumerated index for asset and liability portions
    lhs = array[:stop_idx1]
    rhs = array[stop_idx1:stop_idx2]
    
    # if either asset or liability side missing, we return None
    if lhs.size == 0 or rhs.size == 0:
        return None
    else:
        return (lhs, rhs, stop_idx1, stop_idx2)

In [3]:
def lineItems(vector:np.ndarray, df:pd.DataFrame):
    """
    Retrieving balance sheet information line item names for s3 files
    ------------------------------------------------------------------------------------
    :param vector: (type numpy.ndarray)
        An array of file names for .csv files from s3 to iterate through
        
    :return: (type tuple)
        Return a tuple of arrays; left is the asset items, and right is the liability & equity items 
    """
    
    # retrieve the asset and liability & equity terms from the dataframe
    response = bsSplit(vector)
    print(vector)
    
    # if response is present we continue 
    if response is not None:
        lhs, rhs, index1, index2 = response      # decompose response object to retrieve index
        
        print('\tAsset Lineitems')
        print(lhs)
        print('\tLiability & Equity Lineitems')
        print(rhs)
        
        # save contents of asset and liability splits to AWS S3 bucket
        dfA = df.iloc[:index1]                  # asset dataframe
        dfL = df.iloc[index1:index2]            # liability and equity dataframe
        
        return (dfA, dfL)
    else:
        return None
   

## Final Main Execution

In [4]:
if __name__ == "__main__":
    
    # initiate s3 bucket and corresponding data folder
    bucket = "ran-s3-systemic-risk"
    
    pdf_data_folder ="Output/X-17A-5-CLEAN-PDFS/"
    png_data_folder ="Output/X-17A-5-CLEAN-PNGS/"
    
    pdf_output_folder = 'Output/X-17A-5-SPLIT-PDFS/'
    png_output_folder = 'Output/X-17A-5-SPLIT-PNGS/'
        
    # Amazon Textract client and Sagemaker session
    s3 = boto3.client('s3')
    session = Session()
    
    # discover all of the data that you want to parse (both PDFS and PNGS)
    pdf_paths = np.array(session.list_s3_files(bucket, pdf_data_folder))[1:]
    pdf_asset_split = np.array(session.list_s3_files(bucket, pdf_output_folder + 'Assets/'))
    pdf_liability_split = np.array(session.list_s3_files(bucket, pdf_output_folder + 'Liability & Equity/'))
    
    png_paths = np.array(session.list_s3_files(bucket, png_data_folder))[1:]
    png_asset_split = np.array(session.list_s3_files(bucket, png_output_folder + 'Assets/'))
    png_liability_split = np.array(session.list_s3_files(bucket, png_output_folder + 'Liability & Equity/'))
    
    # --------------------------------------------------------------------------------------------------
    # PDF PROCESSING (LINE-ITEM SPLIT)
    # --------------------------------------------------------------------------------------------------
    
    # iterate through files from s3 bucket 
    for file in pdf_paths:
        print('\n', file)
        fileName = file.split('/')[-1]                                          # file-name for a given path
        asset_name = pdf_output_folder + 'Assets/' + fileName                   # export path to assets
        liability_name = pdf_output_folder + 'Liability & Equity/' + fileName   # export path to liability and equity
        
        # check to see presence of split files 
        if (asset_name not in pdf_asset_split) or (liability_name not in pdf_liability_split) or True:
        
            # download temporary file from s3 bucket
            s3.download_file(bucket, file, 'temp.csv')
            df = pd.read_csv('temp.csv')

            n = df.columns.size   # the number of columns in read dataframe    

            if n > 1: # if there is more than 1 column we continue examination 

                # all line item for balance sheet (first column)
                arr = df[df.columns[0]].dropna().values     

                # extract line items if possible for both asset and liability terms
                response = lineItems(arr, df)

                # if response not None we decompose each
                if response is not None:
                    # unpack the response object to component parts
                    df_asset, df_liability = response

                    # writing data frame to .csv file (we overwrite file name to save space)
                    df_asset.to_csv(fileName, index=False)
                    with open(fileName, 'rb') as data:
                        s3.put_object(Bucket=bucket, Key=asset_name, Body=data)

                    df_liability.to_csv(fileName, index=False)
                    with open(fileName, 'rb') as data:
                        s3.put_object(Bucket=bucket, Key=liability_name, Body=data)

                    # remove local file after it has been created
                    os.remove(fileName)

            else:
                print('{} incomplete dataframe'.format(file))
                
        else:
            print("We've already downloaded {}".format(fileName))
    
    # --------------------------------------------------------------------------------------------------
    # PNG PROCESSING (LINE-ITEM SPLIT)
    # --------------------------------------------------------------------------------------------------
    
    # iterate through files from s3 bucket 
    for file in png_paths:
        print('\n', file)
        fileName = file.split('/')[-1]                                          # file-name for a given path
        asset_name = png_output_folder + 'Assets/' + fileName                   # export path to assets
        liability_name = png_output_folder + 'Liability & Equity/' + fileName   # export path to liability and equity
        
        # check to see presence of split files 
        if (asset_name not in png_asset_split) or (liability_name not in png_liability_split) or True:
        
            # download temporary file from s3 bucket
            s3.download_file(bucket, file, 'temp.csv')
            df = pd.read_csv('temp.csv')

            n = df.columns.size   # the number of columns in read dataframe    

            if n > 1: # if there is more than 1 column we continue examination 

                # all line item for balance sheet (first column)
                arr = df[df.columns[0]].dropna().values     

                # extract line items if possible for both asset and liability terms
                response = lineItems(arr, df)

                # if response not None we decompose each
                if response is not None:
                    # unpack the response object to component parts
                    df_asset, df_liability = response

                    # writing data frame to .csv file (we overwrite file name to save space)
                    df_asset.to_csv(fileName, index=False)
                    with open(fileName, 'rb') as data:
                        s3.put_object(Bucket=bucket, Key=asset_name, Body=data)

                    df_liability.to_csv(fileName, index=False)
                    with open(fileName, 'rb') as data:
                        s3.put_object(Bucket=bucket, Key=liability_name, Body=data)

                    # remove local file after it has been created
                    os.remove(fileName)

            else:
                print('{} incomplete dataframe'.format(file))
                
        else:
            print("We've already downloaded {}".format(fileName))
    
    # remove local file for storing cleaned data  
    os.remove('temp.csv')
    print('\nAll X-17A-5 files have been downloaded and split between Asset and Liability & Equity terms')


 Output/X-17A-5-CLEAN-PDFS/1224385-2004-03-01.csv
['Cash' 'Cash segregated pursuant to federal regulations'
 'Securities purchased under agreements to resell'
 'Securities owned, at market value ($8,769,300 pledged as collateral)'
 'Receivable from broker-dealers and clearing organizations'
 'Receivable from customers' 'Accrued interest receivable'
 'Property, equipment, and leasehold improvements, net' 'Goodwill, net'
 'Other assets' 'Total assets'
 'Securities sold under agreements to repurchase'
 'Securities sold, not yet purchased, at market value'
 'Payable to broker-dealers and clearing organizations'
 'Payable to customers' 'Accrued interest payable' 'Other liabilities'
 'Total liabilities' 'Subordinated borrowings' "Member's contributions"
 'Accumulated earnings' "Total member's equity"
 "Total liabilities and member's equity"]
	Asset Lineitems
['Cash' 'Cash segregated pursuant to federal regulations'
 'Securities purchased under agreements to resell'
 'Securities owned, at ma


 Output/X-17A-5-CLEAN-PDFS/1224385-2010-03-12.csv
['Cash' 'Cash segregated pursuant to federal regulations'
 'Financial instruments owned, at fair value ($7,693,394 pledged as collateral)'
 'Receivable from broker-dealers and clearing organizations'
 'Securities purchased under agreements to resell' 'Goodwill'
 'Receivable from customers'
 'Property, equipment, and leasehold improvements, net' 'Other assets'
 'Total assets' 'Securities sold under agreements to repurchase'
 'Financial instruments sold, not yet purchased, at fair value'
 'Payable to broker-dealers and clearing organizations'
 'Payable to customers' 'Other liabilities' 'Total liabilities'
 'Subordinated borrowings' "Member's contributions" 'Accumulated loss'
 "Total member's equity" "Total liabilities and member's equity"]
	Asset Lineitems
['Cash' 'Cash segregated pursuant to federal regulations'
 'Financial instruments owned, at fair value ($7,693,394 pledged as collateral)'
 'Receivable from broker-dealers and clearing


 Output/X-17A-5-CLEAN-PDFS/1224385-2017-03-01.csv
['Cash' 'Cash segregated pursuant to federal regulations'
 'Financial instruments owned, at fair value ($29,976,201 pledged as collateral)'
 'Securities borrowed' 'Securities purchased under agreements to resell'
 'Receivable from broker-dealers and clearing organizations'
 'Receivable from customers' 'Goodwill'
 'Property, equipment, and leasehold improvements, net' 'Other assets'
 'Total assets' 'Securities sold under agreements to repurchase'
 'Financial instruments sold, not yet purchased, at fair value'
 'Securities loaned' 'Payable to customers'
 'Payable to broker-dealers and clearing organizations'
 'Other liabilities' 'Total liabilities' 'Subordinated borrowings'
 "Member's contributions" 'Accumulated earnings' "Total member's equity"
 "Total liabilities and member's equity"]
	Asset Lineitems
['Cash' 'Cash segregated pursuant to federal regulations'
 'Financial instruments owned, at fair value ($29,976,201 pledged as collatera


 Output/X-17A-5-CLEAN-PDFS/42352-2003-01-28.csv
['and other regulations'
 'Receivables from brokers, dealers and clearing organizations'
 'Receivables from customers and counterparties' 'Securities borrowed'
 'Securities purchased under agreements to resell'
 'Financial instruments owned, at fair value'
 'Financial instruments owned and pledged as collateral, at fair value'
 'Total financial instruments owned, at fair value' 'Other assets'
 'Short-term borrowings'
 'Payables to brokers, dealers and clearing organizations'
 'Payables to customers and counterparties' 'Securities loaned'
 'Securities sold under agreements to repurchase'
 'Financial instruments sold, but not yet purchased, at fair value'
 'Other liabilities and accrued expenses' 'Long-term borrowings'
 'Subordinated borrowings' "Partners' capital"
 'Accumulated other comprehensive income']
	Asset Lineitems
['and other regulations'
 'Receivables from brokers, dealers and clearing organizations'
 'Receivables from customers


 Output/X-17A-5-CLEAN-PDFS/42352-2010-03-01.csv
['Cash and cash equivalents'
 'Cash and securities segregated for regulatory and other purposes (includes $9,739 at fair value)'
 'Securities borrowed (includes $79,137 at fair value)'
 'Financial instruments purchased under agreements to resell, at fair value'
 'Receivables from brokers, dealers and clearing organizations'
 'Receivables from customers and counterparties (includes $440 at fair value)'
 'Financial instruments owned, at fair value'
 'Financial instruments owned and pledged as collateral, at fair value'
 'Total financial instruments owned, at fair value' 'Other assets'
 'Total assets' 'long-term borrowings (includes $3 at fair value)'
 'Securities loaned (includes $23,810 at fair value)'
 'Financial instruments sold under agreements to repurchase, at fair value'
 'Other secured financings (includes $853 at fair value)'
 'Payables to brokers, dealers and clearing organizations'
 'Payables to customers and counterparties'
 'F

...

	Liability & Equity Lineitems
['Affiliates' 'Other' 'Securities sold under agreements to repurchase'
 'Deposits received for securities loaned'
 'Financial instruments sold, not yet purchased,'
 'U.S. government and government agency securities'
 'Corporate debt securities' 'Equity securities' 'Contractual commitments'
 'Non-U.S. government and government agency securities'
 'Other financial instruments' 'Payables and accrued liabilities:'
 'Customers' 'Brokers, dealers and clearing organizations' 'Affiliates'
 'Other' 'Notes payable to SSBH' 'Subordinated indebtedness to SSBH'
 'Total liabilities' 'authorized, issued and outstanding)'
 'Additional paid-in capital' 'Retained earnings'
 'Accumulated changes in equity from nonowner sources'
 "Total stockholder's equity" "Total liabilities and stockholder's equity"]

 Output/X-17A-5-CLEAN-PNGS/91154-2004-02-27.csv
['Cash and cash equivalents'
 'other regulations or deposited with clearing organizations'
 'Securities purchased under agreem

['Cash and cash equivalents'
 'Cash segregated and on deposit for Federal and other regulations or deposited with clearing organizations'
 'Securities purchased under agreements to resell (including $46,791 at fair value)'
 'Deposits paid for securities borrowed'
 'Financial instruments owned and contractual commitments, at fair value:'
 'Corporate debt securities' 'Equity securities'
 'U.S. government and government agency securities'
 'State and municipal securities'
 'Mortgage loans and collateralized mortgage obligations'
 'Contractual commitments' 'Money market instruments'
 'Foreign government securities' 'Other financial instruments'
 'Receivables:' 'Customers' 'Brokers, dealers and clearing organizations'
 'Other'
 'Property, equipment and leasehold improvements, net of accumulated depreciation and amortization of $928'
 'Goodwill' 'Intangibles' 'Other assets' 'Total assets' 'Affiliates'
 'Other'
 'Securities sold under agreements to repurchase (including $140,683 at fair value

['Cash and cash equivalents'
 'Cash segregated and on deposit for Federal and other regulations or deposited with clearing organizations'
 'Securities purchased under agreements to resell (including $51,797 at fair value)'
 'Deposits paid for securities borrowed'
 'Trading account assets (approximately $20 billion were pledged to'
 'Mortgage-backed securities' 'U.S. Treasury and federal agencies'
 'Corporate' 'Equity securities' 'State and municipal securities'
 'Derivatives' 'Foreign government securities' 'Asset-backed securities'
 'Other debt securities'
 'Investment in Morgan Stanley Smith Barney Holdings LLC (including Preferred Securities of $2,000)'
 'Customers' 'Brokers, dealers and clearing organizations' 'Other'
 'Property, equipment and leasehold improvements, net of'
 'accumulated depreciation and amortization of $884' 'Goodwill'
 'Intangibles' 'Other assets' 'Total assets' 'Affiliates' 'Other'
 'Collateralized short-term financing agreements:'
 'Securities sold under agree


 Output/X-17A-5-CLEAN-PNGS/91154-2016-03-01.csv
['Cash and cash equivalents'
 'Cash segregated under federal and other regulations'
 'Securities borrowed or purchased under agreements to resell (including $102,691 at fair value)'
 'Mortgage-backed securities'
 'U.S. Treasury and federal agency securities' 'Equity securities'
 'Corporate debt securities' 'Asset-backed securities' 'Derivatives'
 'Foreign government securities' 'State and municipal securities'
 'Securities received as collateral, at fair value (all pledged to counterparties)'
 'Customers' 'Brokers, dealers and clearing organizations' 'Other'
 'Property, equipment and leasehold improvements, net of accumulated depreciation and amortization of $1,531'
 'Goodwill' 'Other assets' 'Total assets' 'Short-term borrowings'
 'Securities loaned or sold under agreements to repurchase (including $8,793 at fair value)'
 'U.S. Treasury and federal agency securities' 'Corporate debt securities'
 'Equity securities' 'Derivatives' 'Foreig


 Output/X-17A-5-CLEAN-PNGS/91154-2020-03-02.csv
['Cash' 'Cash segregated under federal and other regulations'
 'Securities borrowed and purchased under agreements to resell (including $102,240 at fair value)'
 'Mortgage-backed securities'
 'U.S. Treasury and federal agency securities' 'Equity securities'
 'Corporate debt securities' 'Asset-backed securities'
 'State and municipal securities' 'Derivatives'
 'Foreign government securities'
 'Securities received as collateral, at fair value (all pledged to counterparties)'
 'Customers' 'Brokers, dealers and clearing organizations' 'Other'
 'Goodwill' 'Other assets (including $1,490 at fair value)' 'Total assets'
 'Short-term borrowings'
 'Securities loaned and sold under agreements to repurchase (including $5,207 at fair value)'
 'U.S. Treasury and federal agency securities' 'Equity securities'
 'Corporate debt securities' 'Derivatives' 'Foreign government securities'
 'Other debt securities' 'Customers'
 'Obligations to return securitie

In [5]:
# # runnning singular analysis on a particular dataset
# # s3.download_file('ran-s3-systemic-risk', 'Output/X-17A-5-CLEAN-PNGS/782124-2014-03-05.csv', 'temp.csv')
# # s3.download_file('ran-s3-systemic-risk', 'Output/X-17A-5-CLEAN-PDFS/42352-2003-01-28.csv', 'temp.csv')
# s3.download_file('ran-s3-systemic-risk', 'Output/X-17A-5-CLEAN-PDFS/1224385-2004-03-01.csv', 'temp.csv')

# df = pd.read_csv('temp.csv')
# os.remove('temp.csv')

# # all line item for balance sheet (first column)
# arr = df[df.columns[0]].dropna().values     

# # extract line items if possible for both asset and liability terms
# response = lineItems(arr, df)